In [ ]:
%matplotlib widget
from bmcs_shell.api import WBCell5ParamXur, WBCell5ParamV2_2Betas, WBCell5ParamXur, \
WBCell5ParamV2, WBCell5ParamV3, WBCell4Param, WBGeoUtils, WBCell2Gammas, WBCell5ParamV4
import numpy as np
from numpy import sqrt, sin, cos, tan, arctan

# Demo showing the kinematics of different waterbomb cells

## Waterbomb cell with 4 parameters (symmetric cell)

In [ ]:
wb = WBCell4Param()
wb.interact()

In [ ]:
WBGeoUtils.export_obj_file(wb, 'wb4_gamma_0.705.obj')

## Waterbomb cells with 5 parameters (asymmetric cell)

### Cell V1

In [ ]:
wb = WBCell5ParamXur()
wb.interact()

### Cell V2

In [ ]:
wb = WBCell5ParamV2()
wb.interact()

### Cell V2 - 2 Betas

In [ ]:
wb = WBCell5ParamV2_2Betas()
wb.interact()

### Cell V3

In [ ]:
wb = WBCell5ParamV3(debug=True)
wb.interact()

In [ ]:
diff = wb.X_Ia[5]-wb.X_Ia[0]
print(diff)
np.sqrt(np.einsum('a, a', diff, diff))

### Cell V4

In [ ]:
wb = WBCell5ParamV4(symmetric=False)
wb.interact()

### Cell 2 Gammas

In [ ]:
wb = WBCell2Gammas()
wb.interact()